Hello and welcome.

This is just a little try to work on this kannada-MNIST challenge. I used a CNN I wrote a while ago. 

Imports

In [ ]:
import os

import numpy as np
import pandas as pd

from tensorflow.keras.utils import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
 

Read and shape data

In [ ]:
x_data = np.genfromtxt('../input/Kannada-MNIST/train.csv', delimiter=',', skip_header=1, usecols=np.arange(1, 785))

#create train picture data
slices_size =int((len(x_data)*0.85))
x_train = x_data[:slices_size]
x_train = x_train.reshape(slices_size, 28, 28)

#create test picture data
slices_size_test = len(x_data) - slices_size
x_test = x_data[slices_size:]
x_test = x_test.reshape(slices_size_test, 28, 28)

#create train labels
y_data = np.genfromtxt('../input/Kannada-MNIST/train.csv', delimiter=',', skip_header=1, usecols=0)
y_train = y_data[:slices_size]
y_test = y_data[slices_size:]


train_size, test_size = x_train.shape[0], x_test.shape[0]

CNN

In [ ]:
#Settings 
width = 28
height = 28
depth = 1
num_classes = 10
epochs = 7

x_train = x_train.reshape(train_size, width, height, depth)
x_test = x_test.reshape(test_size, width, height, depth)
y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)

# Define the DNN
input_img = Input(shape=(width, height, depth))

x = Conv2D(filters=16, kernel_size=3, padding='same')(input_img)
x = Activation("relu")(x)
x = Conv2D(filters=32, kernel_size=3, padding='same')(x)
x = Activation("relu")(x)
x = MaxPool2D()(x)

x = Conv2D(filters=32, kernel_size=3, padding='same')(x)
x = Activation("relu")(x)
x = Conv2D(filters=64, kernel_size=3, padding='same')(x)
x = Activation("relu")(x)
x = MaxPool2D()(x)

x = Flatten()(x)

x = Dense(64)(x)
x = Activation("relu")(x)
x = Dense(num_classes)(x)
output_pred = Activation("softmax")(x)

model = Model(inputs=[input_img], outputs=[output_pred])

model.summary()

# Train the DNN
lr = 5e-4
optimizer = RMSprop(lr=lr)

model.compile(
    loss="categorical_crossentropy",
    optimizer=optimizer,
    metrics=["accuracy"])

model.fit(
    x=x_train,
    y=y_train,
    verbose=1,
    batch_size=64,
    epochs=epochs,
    validation_data=[x_test, y_test])

# Test the DNN
score = model.evaluate(x_test, y_test, verbose=0)
print("Score: ", score)

Submission

In [ ]:
x_sub = np.genfromtxt('../input/Kannada-MNIST/test.csv', delimiter=',', skip_header=1, usecols=np.arange(1, 785))
x_sub = x_sub.reshape(len(x_sub), 28, 28)
predict = model.predict(x_sub)
predict = np.argmax(predict, axis=1)


# create submission file
submission = pd.read_csv('../input/Kannada-MNIST/sample_submission.csv')
submission['label'] = predict
submission.to_csv('submission.csv', index=False)

submission
